In [63]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import matplotlib.pyplot as plt
import cv2  # Para manejar imágenes nuevas
from sklearn.model_selection import train_test_split

In [65]:
# Leer el archivo CSV
data = pd.read_csv('age_gender.csv')

In [66]:
# Convertir píxeles en arrays de numpy
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype="float32"))

In [67]:
# Preparar los datos
X = np.array(data['pixels'].tolist())
X = X.reshape(X.shape[0], 48, 48, 1) / 255.0  # Normalizar los píxeles

In [68]:
data.head()

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,"[129.0, 128.0, 128.0, 126.0, 127.0, 130.0, 133..."
1,1,2,0,20161219222752047.jpg.chip.jpg,"[164.0, 74.0, 111.0, 168.0, 169.0, 171.0, 175...."
2,1,2,0,20161219222832191.jpg.chip.jpg,"[67.0, 70.0, 71.0, 70.0, 69.0, 67.0, 70.0, 79...."
3,1,2,0,20161220144911423.jpg.chip.jpg,"[193.0, 197.0, 198.0, 200.0, 199.0, 200.0, 202..."
4,1,2,0,20161220144914327.jpg.chip.jpg,"[202.0, 205.0, 209.0, 210.0, 209.0, 209.0, 210..."


In [69]:
y = data['age']  # Variable objetivo: edad

In [70]:
# Crear etiquetas para rango de edad
y_min = data['age'] - 2  # Edad mínima estimada (2 años menos)
y_max = data['age'] + 2  # Edad máxima estimada (2 años más)
y_range = np.vstack((y_min, y_max)).T  # Combinar en una sola matriz

In [71]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y_range, test_size=0.22, random_state=37
)

In [72]:
# Definir el modelo de red neuronal convolucional
model = tf.keras.Sequential([
    L.InputLayer(input_shape=(48, 48, 1)),
    L.Conv2D(32, (3, 3), activation='relu'),  # 1 CV
    L.BatchNormalization(),  # 1 BT NM
    L.MaxPooling2D((2, 2)),  # 1 MP
    L.Conv2D(64, (3, 3), activation='relu'),  # 2 CV
    L.MaxPooling2D((2, 2)),  # 2 MP
    L.Conv2D(128, (3, 3), activation='relu'),  # 3 CV
    L.MaxPooling2D((2, 2)),  # 3 MP
    L.Flatten(),
    L.Dense(64, activation='relu'),  # 64 Neuronas
    L.Dropout(rate=0.5),  # 2 NM DP
    L.Dense(2, activation='linear')  # Predicción: Edad mínima y máxima
])

/Users/antonio/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [73]:
# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)

In [74]:
# Callback para detener el entrenamiento temprano
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_loss') < 110:
            print("\nReached 110 val_loss so cancelling training!")
            self.model.stop_training = True

callback = myCallback()

In [76]:
# Entrenamiento del modelo
history = model.fit(
    X_train, y_train, epochs=20, validation_split=0.1, batch_size=64, callbacks=[callback]
)

Epoch 1/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step - loss: 149.8247 - mae: 9.1378 - val_loss: 201.9528 - val_mae: 11.3062
Epoch 2/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 140.8590 - mae: 8.7641 - val_loss: 154.2560 - val_mae: 8.9957
Epoch 3/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 127.7072 - mae: 8.3658
Reached 110 val_loss so cancelling training!
260/260 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step - loss: 127.7069 - mae: 8.3658 - val_loss: 107.0964 - val_mae: 7.8166


In [77]:
# Evaluar el modelo
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Mean Squared Error: {mse}')
print(f'Test Mean Absolute Error: {mae}')

Test Mean Squared Error: 97.41747283935547
Test Mean Absolute Error: 7.51378870010376


In [78]:
# Guardar el modelo entrenado
model.save('age_prediction_model.h5')
print("Modelo guardado como 'age_prediction_model.h5'")


Modelo guardado como 'age_prediction_model.h5'


In [84]:
# Función para cargar el modelo y predecir

def predict_age_1(image_path, model_path):
    try:
        # Cargar el modelo
        loaded_model = tf.keras.models.load_model(model_path)
        print("Modelo cargado correctamente.")
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        return None

    try:
        # Preprocesar la imagen
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Error: No se pudo cargar la imagen desde {image_path}.")
            return None
        img = cv2.resize(img, (48, 48))  # Redimensionar a 48x48
        img = img.reshape(1, 48, 48, 1) / 255.0  # Normalizar
    except Exception as e:
        print(f"Error al procesar la imagen: {e}")
        return None

    try:
        # Predecir la edad
        predicted_age = loaded_model.predict(img)
        print(f'Edad estimada: {predicted_age[0][0]:.2f} años normal')
        return predicted_age[0][0]
    except Exception as e:
        print(f"Error durante la predicción: {e}")
        return None

In [86]:
# Función para predecir rango de edad
def predict_age_range(image_path, model_path='age_range_prediction_model.h5'):
    # Cargar el modelo
    loaded_model = tf.keras.models.load_model(model_path)
    print("Modelo cargado correctamente.")

    # Cargar la imagen y convertirla a escala de grises
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: No se pudo cargar la imagen desde {image_path}.")
        return None

    # Usar un detector de rostros para recortar automáticamente el rostro
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        print("No se detectó ningún rostro en la imagen.")
        return None

    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]  # Recortar el rostro
        face = cv2.resize(face, (48, 48))  # Redimensionar a 48x48
        face = face.reshape(1, 48, 48, 1) / 255.0  # Normalizar
        break  # Solo se toma el primer rostro

    # Hacer la predicción
    predicted_range = loaded_model.predict(face)
    age_min, age_max = predicted_range[0]
    print(f'Rango de edad estimado: {age_min:.1f} - {age_max:.1f} años')
    return age_min, age_max

In [ ]:
image_path = 'Yooo.jpg'
model_path = 'age_prediction_model.h5'

predict_age(image_path, model_path)
predict_age_range(image_path, model_path)
#predict_age_CV(image_path, model_path)

Modelo cargado correctamente.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


Edad estimada: 35.06 años normal
Modelo cargado correctamente.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Edad estimada: 19.80 años con Cascade


19.79531